In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sympy import diff, symbols


In [14]:
tr_data = pd.read_csv('./../data/mnist_train.csv')
ts_data = pd.read_csv('./../data/mnist_test.csv')

In [15]:
tr_data.describe()


,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
count,60000.000000,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,...,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.0000,60000.0,60000.0,60000.0,60000.0
mean,4.453933,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.200433,0.088867,0.045633,0.019283,0.015117,0.0020,0.0,0.0,0.0,0.0
std,2.889270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.042472,3.956189,2.839845,1.686770,1.678283,0.3466,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.000000,254.000000,62.0000,0.0,0.0,0.0,0.0


In [16]:
train_data = tr_data.drop(columns='label').values
train_labels = tr_data.label.values
test_data = ts_data.drop(columns='label').values
test_labels = ts_data.label.values

In [19]:
print(train_data.shape)
print(train_labels.shape)

(60000, 784)
(60000,)


In [21]:
train_data = train_data/255
test_data = test_data/255

In [76]:
class Activation:
    def softmax(self, z):
        temp = np.sum(np.exp(z)) 
        return np.exp(z) / temp
    
    def relu(self, z):
        return np.maximum(0, z)
    
    def sigmoid(self, z):
        return 1/(1 + np.exp(-z))
    
    def use(self, string):
        match string:
            case 'relu':
                return self.relu
            case 'sigmoid':
                return self.sigmoid
            case 'softmax':
                return self.softmax
            case default:
                return z
    

In [78]:
activation = Activation().use('softmax')
print(activation(np.array([-1,3,4])))

[0.00490169 0.26762315 0.72747516]


In [70]:
class DenseLayer:
    def __init__(self, units, a_in, activation=None):
        self.units = units 
        self.a_in = a_in
        self.activation = activation
        self.w = np.zeros(shape=(len(a_in), units)) #!needs modification
        self.b = np.zeros(units)

    def __str__(self):
        return f'''
        Number of units: {self.units}
        Input shape(a_in.shape): {self.a_in.shape}
        weight has shape(w.shape): {self.w.shape}
        bias has shape(b.shape): {self.b.shape}
        '''

    def output(self):
        if(self.activation == None): return self.a_in@self.w + self.b
        return self.activation(self.a_in@self.w + self.b)

        

In [71]:
x = np.array([200, 17])
layer = DenseLayer(2, x)
print(layer)
print(layer.output())


        Number of units: 2
        Input shape(a_in.shape): (2,)
        weight has shape(w.shape): (2, 2)
        bias has shape(b.shape): (2,)
        
[0. 0.]


[0. 0.]
